<h1> Assignment #2 </h1>

You have to move several cargos across the gridworld into a common desirable rectangle area. There are several cargos, you can move each of them either horizontally or vertically by one cell up or down. Size of the overall world may vary, as well as placement of the cargos and desirable area. The game ends when all the cargos are in the desirable area and do not overlap.

Reward function: 
* for each cell of cargo placed in desirable area in the end of the turn, reward is $+1$
* for each cell where cargos overlap in the end of the turn, reward is $-1$

---

Submit `{name}_{surname}.zip` file containing:
* `{name}_{surname}.ipynb` with code for learning your model using your realisation of environment
* `{name}_{surname}.py` script with `complete_task(path_to_infile, path_to_outfile)` function
* report `{name}_{surname}.pdf`, containing information on:
  * choice of the environment representation used for the model training
  * way of training environment generation
  * architecture choice for the neural network you use for the prediction

## Input of the script

`{infile}.txt` file with the field description. Elements of the field are separated by space. For example: 

```
0 0 2 2 0
0 r r r 0
0 r r r 1
0 r r r 1
0 0 0 0 1
```

* `0` - blank space, we may move objects here
* `r` - desirable area, we should move objects here
* `1`, `2`, ... - actual object shapes, does not change, moved as a solid object

In your output file, you have to record lines in the following manner: `{id} {D/U/R/L}`. For the given example, possible sequence of steps is:
```
2 D
1 L
1 U
2 L
```

Here the rewards are: 
1. +2 for 2 cells of `2`
2. +4 for 2 cells of `2` and 2 cells of `1`
3. +3 (= +4 - 1) for 2 cells of `2` and 2 cells of `1` and -1 overlapping cell 
4. **The end**

## Output of the script

Sequence of actions, written in `{outfile}.txt`. Rewrite it after each 100th action, please, and after you rich the final position.

In [ ]:
score = []
for episode in range(MAX_EPISODES):
    env.reset()
    total_rewards = []
    for cargo in env.cargos:
        current_state = np.array(env.goal_distance(cargo))
        transitions = []
        done = False
        t_lens = []
        for step in range(MAX_STEPS):
            step_reward = -1

            action = model.act(current_state, EPS_START)
            prev_state = current_state

            if env.check_move(cargo, actions[action]):
                env.move_cargo(cargo, actions[action])
                current_state = np.array(env.goal_distance(cargo))
            else:
                step_reward = -10

            other_cargo_overlaps = env.get_cargo_overlaps()
            desired_cargo_overlaps = env.get_cargo_overlaps_with_desired()
            step_reward -= other_cargo_overlaps
            step_reward += desired_cargo_overlaps
            transitions.append((prev_state, action, step_reward))

            cargo_num_cells = len(cargo)
            desired_num_cells = len(env.desired_space)
            if desired_cargo_overlaps == cargo_num_cells:
                done = True
                break

            score.append(len(transitions))
            reward_batch = torch.Tensor([r for (s, a, r) in
                                         transitions]).flip(dims=(0,))

            total_rewards.append(reward_batch.mean().item())

            batch_Gvals = []

            for i in range(len(transitions)):
                new_G = torch.sum(reward_batch[i:] *
                                  torch.Tensor([gamma ** j for j in
                                                range(len(transitions) - i)]))
                batch_Gvals.append(new_G)
            expected_returns_batch = torch.FloatTensor(batch_Gvals)

            expected_returns_batch /= expected_returns_batch.max()

            state_batch = torch.Tensor(np.array([s for (s, a, r) in transitions]))
            action_batch = torch.Tensor(np.array([a for (s, a, r) in transitions]))

            pred_batch = model(state_batch)
            prob_batch = pred_batch.gather(dim=1, index=action_batch
                                           .long().view(-1, 1)).squeeze()

            loss = F.mse_loss(prob_batch, expected_returns_batch)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            if EPS_START > EPS_END and episode > (MAX_EPISODES / 2):
                EPS_START *= EPS_DECAY

        if episode % 20 == 0:
            print('Cargo', cargo)
            print("Episode: {}, Avg Reward: {}".format(episode, np.mean(total_rewards)))
            print('loss', loss.item())
            print(reward_batch)
            actions_taken = [actions[a] for (s, a, r) in transitions]
            print('actions', actions_taken)